In [ ]:

import sys,os
import numpy as np
import pandas as pd
from binance.client import Client
import time
import progressbar
import matplotlib
from matplotlib import cm
import matplotlib.pyplot as plt
from binance.enums import *
import mpl_finance
# from BinanceKeys import BinanceKey1
# sys.path.append('C:\\Users\\Admin\\python-binance-master\\examples')

# os.chdir('C:/Users/Admin/python-binance-master/examples')
# sys.path.append('C:/Users/Admin/python-binance-master/examples')

import site
site.addsitedir(r'C:\\Users\\Admin\\python-binance-master\\examples')

import save_historical_data_Roibal
from BinanceKeys import BinanceKey1
# import technical_indicators

api_key = BinanceKey1['api_key']
api_secret = BinanceKey1['api_secret']

client = Client(api_key, api_secret)

# ================ Define variable values =================
watch_list = ['ETHBTC','ADABTC','EOSBTC','XRPBTC']

start_date = "1 month ago UTC"
kline = Client.KLINE_INTERVAL_5MINUTE

ma_length = 126
# =========================================================

# Test Orders first to check

#Define Market Orders for ETH - Buy and sell
# buy_order_eth = client.order_market_buy(symbol='ETHBTC',quantity=0.01)
# sell_order_eth = client.order_market_sell(symbol='ETHBTC',quantity=0.01)

# ================== Main Function ====================
def run():
    # get system status
    #Create List of Crypto Pairs to Watch
#     list_of_symbols = ['BTCUSDT', 'ETHUSDT', 'BNBUSDT','BNBBTC', 'ETHBTC', 'LTCBTC']

    print("\n\n----------------------------------------------------------------------\n\n")
    print("Welcome to the Binance Bot Python Script\nCreated 2019 by Manoj Shenoy\n")
    print("You can also find this on github- https://www.github.com/manoj-shenoy")
    print("                                  https://www.github.com/stevens-morons")
    
    time.sleep(5)
#     try:
#         #Example Visualizations of Coins
        
#         for coin in watch_list:
            
#             save_historical_data_Roibal.save_historic_klines_csv(coin,
#                                                                  "1 month ago UTC", "now UTC",
#                                                                  Client.KLINE_INTERVAL_4HOUR)
#     except():
#         pass
    
    #Get Status of Exchange & Account
    try:
        status = client.get_system_status()
        print("\nExchange Status: ", status)

        #Account Withdrawal History Info
#         withdraws = client.get_withdraw_history()
#         print("\nClient Withdraw History: ", withdraws)

        #get Exchange Info
#         info = client.get_exchange_info()
#         print("\nExchange Info (Limits): ", info)
    
    except():
        pass

    # place a test market buy order, to place an actual order use the create_order function
    # if '1000 ms ahead of server time' error encountered, visit https://github.com/sammchardy/python-binance/issues/249
#     try:
#         order = client.create_test_order(
#             symbol='ETHBTC',
#             side=Client.SIDE_BUY,
#             type=Client.ORDER_TYPE_MARKET,
#             quantity=0.0001)
#     except:
#         print("\n \n \nATTENTION: NON-VALID CONNECTION WITH BINANCE \n \n \n")

    #Get Info about Coins in Watch List
    coin_prices(watch_list=watch_list)
    coin_tickers(watch_list=watch_list)
    
    # Get historical data using Klines DatFrame
    data_eth = create_klines_df(symbol='ETHBTC',start_date=start_date,kline=kline)
    
    # Delete any old files
#     remove_files(csv='Binance_ETHBTC.txt')
#     remove_files(csv='Binance_ADABTC.txt')
#     remove_files(csv='Binance_EOSBTC.txt')
    
    # Save Historical Data for Coins
#     save_coin_files(start_date=start_date,kline=kline) # Currently ETHBTC, ADABTC, EOSBTC
    

#     data_eth = read_coin_files(csv='Binance_ETHBTC.txt')
    
#     print(data_eth.columns)
#     print('ETH Binance data below\n')
#     print(data_eth)
#     data_ada = read_coin_files(csv='Binance_ADABTC.txt')
#     data_eos = read_coin_files(csv='Binance_EOSBTC.txt')
    
    #for symbol in list_of_symbols:
    #    market_depth(symbol)
    
    # ===== Run all bots here ======
    mabandsbot(historical_data=data_eth,symbol='ETHBTC',ping_interval=300)
#     mabandsbot(historical_data=data_ada,symbol='ADABTC',wait_time=60)
#     mabandsbot(historical_data=data_eos,symbol='EOSBTC',wait_time=60)
    
    try:
        for symbol in watch_list:
            #get recent trades
            trades = client.get_recent_trades(symbol=symbol)
            print("\nRecent Trades: ", trades)
            print("Local Time: ", time.localtime())
            print("Recent Trades Time: ", convert_time_binance(trades[0]['time']))

            #get historical trades
            hist_trades = client.get_historical_trades(symbol=symbol)
            print("\nHistorical Trades: ", hist_trades)
            
            #get aggregate trades
            agg_trades = client.get_aggregate_trades(symbol=symbol)
            print("\nAggregate Trades: ", agg_trades)
    except:
        print('\n \n \nATTENTION: NON VALID CONNECTION WITH BINANCE \n \n \n')


def convert_time_binance(gt):
    #Converts from Binance Time Format (milliseconds) to time-struct
    #From Binance-Trader Comment Section Code
    #gt = client.get_server_time()
    print("Binance Time: ", gt)
    print(time.localtime())
    aa = str(gt)
    bb = aa.replace("{'serverTime': ","")
    aa = bb.replace("}","")
    gg=int(aa)
    ff=gg-10799260
    uu=ff/1000
    yy=int(uu)
    tt=time.localtime(yy)
    #print(tt)
    return tt


def market_depth(sym, num_entries=20):
    #Get market depth
    #Retrieve and format market depth (order book) including time-stamp
    i=0     #Used as a counter for number of entries
    print("Order Book: ", convert_time_binance(client.get_server_time()))
    depth = client.get_order_book(symbol=sym)
    print(depth)
    print(depth['asks'][0])
    bid_tot,ask_tot=0.0
    ask_price =[]
    ask_quantity = []
    bid_price = []
    bid_quantity = []
    place_order_ask_price,place_order_bid_price,max_order_ask,max_order_bid = 0

    print("\n", sym, "\nDepth     ASKS:\n")
    print("Price     Amount")
    for ask in depth['asks']:
        if i<num_entries:
            if float(ask[1])>float(max_order_ask):
                #Determine Price to place ask order based on highest volume
                max_order_ask=ask[1]
                place_order_ask_price=round(float(ask[0]),5)-0.0001
            #ask_list.append([ask[0], ask[1]])
            ask_price.append(float(ask[0]))
            ask_tot+=float(ask[1])
            ask_quantity.append(ask_tot)
            #print(ask)
            i+=1
    j=0     #Secondary Counter for Bids
    print("\n", sym, "\nDepth     BIDS:\n")
    print("Price     Amount")
    for bid in depth['bids']:
        if j<num_entries:
            if float(bid[1])>float(max_order_bid):
                #Determine Price to place ask order based on highest volume
                max_order_bid=bid[1]
                place_order_bid_price=round(float(bid[0]),5)+0.0001
            bid_price.append(float(bid[0]))
            bid_tot += float(bid[1])
            bid_quantity.append(bid_tot)
            #print(bid)
            j+=1
    return ask_price, ask_quantity, bid_price, bid_quantity, place_order_ask_price, place_order_bid_price
    #Plot Data

# def scalping_orders(coin, wait=1, tot_time=1):
#     #Function for placing 'scalp orders'
#     #Calls on Visualizing Scalping Orders Function
#     ap, aq, bp, bq, place_ask_order, place_bid_order, spread, proj_spread, max_bid, min_ask = visualize_market_depth(wait, tot_time, coin)
#     print("Coin: {}\nPrice to Place Ask Order: {}\nPrice to place Bid Order: {}".format(coin,place_ask_order, place_bid_order))
#     print("Spread: {} % Projected Spread {} %".format(spread, proj_spread))
#     print("Max Bid: {} Min Ask: {}".format(max_bid, min_ask))
#     #Place Orders based on calculated bid-ask orders if projected > 0.05% (transaction fee)
#    
#     """
#     if proj_spread > 0.05:
#         quant1=100          #Determine Code Required to calculate 'minimum' quantity
#         #Place Bid Order:
#         bid_order1 = client.order_limit_buy(
#             symbol=coin,
#             quantity=quant1,
#             price=place_bid_order)
#         #Place Ask Order
#         ask_order1 = client.order_limit_sell(
#             symbol=coin,
#             quantity=quant1,
#             price=place_ask_order)


# Place second order if current spread > 0.05% (transaction fee)



# def visualize_market_depth(wait_time_sec='1', tot_time='1', sym='ICXBNB', precision=5):
#     cycles = int(tot_time)/int(wait_time_sec)
#     start_time = time.asctime()
#     fig, ax = plt.subplots()
#     for i in range(1,int(cycles)+1):
#         ask_pri, ask_quan, bid_pri, bid_quan, ask_order, bid_order = market_depth(sym)

#         #print(ask_price)
#         plt.plot(ask_pri, ask_quan, color = 'red', label='asks-cycle: {}'.format(i))
#         plt.plot(bid_pri, bid_quan, color = 'blue', label = 'bids-cycle: {}'.format(i))

#         #ax.plot(depth['bids'][0], depth['bids'][1])
#         max_bid = max(bid_pri)
#         min_ask = min(ask_pri)
#         max_quant = max(ask_quan[-1], bid_quan[-1])
#         spread = round(((min_ask-max_bid)/min_ask)*100,5)   #Spread based on market
#         proj_order_spread = round(((ask_order-bid_order)/ask_order)*100, precision)
#         price=round(((max_bid+min_ask)/2), precision)
#         plt.plot([price, price],[0, max_quant], color = 'green', label = 'Price - Cycle: {}'.format(i)) #Vertical Line for Price
#         plt.plot([ask_order, ask_order],[0, max_quant], color = 'black', label = 'Ask - Cycle: {}'.format(i))
#         plt.plot([bid_order, bid_order],[0, max_quant], color = 'black', label = 'Buy - Cycle: {}'.format(i))
#         #plt.plot([min_ask, min_ask],[0, max_quant], color = 'grey', label = 'Min Ask - Cycle: {}'.format(i))
#         #plt.plot([max_bid, max_bid],[0, max_quant], color = 'grey', label = 'Max Buy - Cycle: {}'.format(i))
#         ax.annotate("Max Bid: {} \nMin Ask: {}\nSpread: {} %\nCycle: {}\nPrice: {}"
#                     "\nPlace Bid: {} \nPlace Ask: {}\n Projected Spread: {} %".format(max_bid, min_ask, spread, i, price, bid_order, ask_order, proj_order_spread),
#                     xy=(max_bid, ask_quan[-1]), xytext=(max_bid, ask_quan[0]))
#         if i==(cycles+1):
#             break
#         else:
#             time.sleep(int(wait_time_sec))
#     #end_time = time.asctime()
#     ax.set(xlabel='Price', ylabel='Quantity',
#        title='Binance Order Book: {} \n {}\n Cycle Time: {} seconds - Num Cycles: {}'.format(sym, start_time, wait_time_sec, cycles))
#     plt.legend()
#     plt.show()
#     return ask_pri, ask_quan, bid_pri, bid_quan, ask_order, bid_order, spread, proj_order_spread, max_bid, min_ask


def coin_prices(watch_list):
    #Will print to screen, prices of coins on 'watch list'
    #returns all prices
    prices = client.get_all_tickers()
    print("\nSelected (watch list) Ticker Prices: ")
    for price in prices:
        if price['symbol'] in watch_list:
            print(price)
    return prices


def coin_tickers(watch_list):
    # Prints to screen tickers for 'watch list' coins
    # Returns list of all price tickers
    tickers = client.get_orderbook_tickers()
    print("\nWatch List Order Tickers: \n")
    for tick in tickers:
        if tick['symbol'] in watch_list:
            print(tick)
    return tickers

def portfolio_management(deposit = '10000', withdraw=0, portfolio_amt = '0', portfolio_type='USDT', test_acct='True'):
    """The Portfolio Management Function will be used to track profit/loss of Portfolio in Any Particular Currency (Default: USDT)"""
    #Maintain Portfolio Statistics (Total Profit/Loss) in a file
    pass

# Determine position size of each asset
def position_sizing(symbol,percent_of_asset):
    asset_balance=client.get_asset_balance(asset=symbol)
    position_size=percent_of_asset * float(asset_balance['free'])
    return position_size

def create_klines_df(symbol,start_date,kline):
    header=['Time','Open','High','Low','Close']
    
    klines=client.get_historical_klines(symbol, kline, start_date, "now UTC")
    ohlc=[]
    for kline in klines:

        time1 = int(kline[0])
        open1 = float(kline[1])
        Low = float(kline[3])
        High = float(kline[2])
        Close = float(kline[4])
        Volume = float(kline[5])
#         format_kline = "{}, {}, {}, {}, {}, {}\n".format(time, open1, High, Low, Close, Volume)
        ohlc.append([time1, open1, Close, High, Low, Volume])
    
    klines=np.array(ohlc)

    klines_reqd=klines[:,0:5]

    klines_df=pd.DataFrame(klines_reqd,columns=header)
#     print(klines_df)
    return klines_df

def remove_files(csv):
    try:
        os.remove(csv)
        print('Old File removed!')
    
    except FileNotFoundError:
        print('The system cannot find the file specified')

# save ETH, ADA and EOS files by running save_historical_data
def save_coin_files(start_date,kline):
    for symbol in watch_list:
        save_historical_data_Roibal.save_historic_klines_csv(symbol, start_date, "now UTC", kline)    
        print('{}-Historical Data File saved '.format(symbol))


def read_coin_files(csv):
    read_data=pd.read_csv(csv, delimiter=',', encoding="utf-8-sig")
#     print(type(read_data))
    return read_data

# data = [data_eth,data_ada,data_eos]
     
#MACD, MACD Signal and MACD difference
def MABands(df, n):
    EMAHigh = pd.Series(df['High'].rolling(n).mean(),name='EMAHigh')
    EMALow = pd.Series(df['Low'].rolling(n).mean(),name='EMALow')

    df = df.join(EMAHigh)
    df = df.join(EMALow)

    return df

def vol_burst():
    pass        
        
def progress():
    bar = progressbar.ProgressBar(maxval=20, \
        widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    for i in range(20):
        bar.update(i+1)
        time.sleep(0.1)
    bar.finish()

n=126    
def mabandsbot(historical_data,symbol,ping_interval=300):
        
    
    # Initialise position
    position=0
    percent_of_asset=0.7
    trade_size = position_sizing(symbol=symbol[:3],
                                 percent_of_asset=percent_of_asset)
    num_cycles = 1e+15
    for i in range(1,int(num_cycles)): 
    #     position=0
        # Delete any old files
#         remove_files(csv='Binance_ETHBTC.txt')
#         remove_files(csv='Binance_ADABTC.txt')
#         remove_files(csv='Binance_EOSBTC.txt')
    
#         Save Historical Data for Coins
#         save_coin_files(start_date=start_date,kline=kline) # Currently ETHBTC, ADABTC, EOSBTC
        
        data_eth = create_klines_df(symbol='ETHBTC',start_date=start_date,kline=kline)
        
        ma_bands=MABands(historical_data,n=ma_length)
        ma_high = ma_bands['EMAHigh']
        ma_low = ma_bands['EMALow']
        close = historical_data['Close']
        
        progress()
        
        if (close.iloc[-1] > ma_high.iloc[-1] and
            close.iloc[-2] < ma_high.iloc[-2] and
            position==0):
            print('Buy Condition Triggered\n')
#             print('{}'.format(symbol)+' Bought at '+ '{}'.format(market_depth(symbol,num_entries=20)[0]))
            client.order_market_buy(symbol=symbol,
                                    quantity=trade_size)
            position+=trade_size # change this to actual position size
            print('{} Qty of {} Bought at {}'.format(trade_size,
                                                     symbol,
                                                     client.get_my_trades(symbol=symbol)[-1]['price']))
        
        elif (close.iloc[-1] < ma_low.iloc[-1] and
              close.iloc[-2] > ma_low.iloc[-2] and
              position==0):
            print('Sell Condition Triggered\n')
#             print('{}'.format(symbol)+' Sold at '+ '{}'.format(market_depth(symbol,num_entries=20)[2]))
            client.order_market_sell(symbol=symbol,
                                     quantity=trade_size)
            position-=trade_size
            print('{} Qty of {} Sold at {}'.format(trade_size,
                                                   symbol,
                                                   client.get_my_trades(symbol=symbol)[-1]['price']))

        if (position > 0 and close.iloc[-1] < ma_low.iloc[-1] and
            close.iloc[-2] > ma_low.iloc[-2]):
            print('Stop-Loss of Buy Position Triggered\n')
#             print('{}'.format(symbol)+' Sold at '+ '{}'.format(market_depth(symbol,num_entries=20)[2]))
            client.order_market_sell(symbol=symbol,
                                     quantity=position)
            position=0
            print('{} Qty of {} Stop-Loss- Sell triggered at {}'.format(position,
                                                                        symbol,
                                                                        client.get_my_trades(symbol=symbol)[-1]['price']))
        
        elif (position < 0 and close.iloc[-1] > ma_high.iloc[-1] and
            close.iloc[-2] < ma_high.iloc[-2]):
            print('Stop-Loss of Sell Position Triggered\n')
#             print('{}'.format(symbol)+' Bought at '+ '{}'.format(market_depth(symbol,num_entries=20)[0]))
            client.order_market_buy(symbol=symbol,
                                    quantity=abs(position))
            position=0
            print('{} Qty of {} Stop-Loss- Sell triggered at {}'.format(position,
                                                                        symbol,
                                                                        client.get_my_trades(symbol=symbol)[-1]['price']))

        else:
            print("No Trading Signal Triggered")

        time.sleep(ping_interval)

"""
#trade aggregator (generator)
agg_trades = client.aggregate_trade_iter(symbol='ETHBTC', start_str='30 minutes ago UTC')
# iterate over the trade iterator
for trade in agg_trades:
    pass
    #print(trade)
    # do something with the trade data

# convert the iterator to a list
# note: generators can only be iterated over once so we need to call it again
agg_trades = client.aggregate_trade_iter(symbol='ETHBTC', start_str='30 minutes ago UTC')
agg_trade_list = list(agg_trades)

# fetch 30 minute klines for the last month of 2017
klines = client.get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_30MINUTE, "1 Dec, 2017", "1 Jan, 2018")
#for kline in klines:
    #print(kline)
"""


if __name__ == "__main__":
    run()




----------------------------------------------------------------------


Welcome to the Binance Bot Python Script
Created 2019 by Manoj Shenoy

You can also find this on github- https://www.github.com/manoj-shenoy
                                  https://www.github.com/stevens-morons

Exchange Status:  {'msg': 'normal', 'status': 0}

Selected (watch list) Ticker Prices: 
{'symbol': 'ETHBTC', 'price': '0.01756500'}
{'symbol': 'EOSBTC', 'price': '0.00034980'}
{'symbol': 'XRPBTC', 'price': '0.00002616'}
{'symbol': 'ADABTC', 'price': '0.00000469'}

Watch List Order Tickers: 

{'symbol': 'ETHBTC', 'bidPrice': '0.01756500', 'bidQty': '0.31900000', 'askPrice': '0.01756700', 'askQty': '0.31300000'}
{'symbol': 'EOSBTC', 'bidPrice': '0.00034970', 'bidQty': '31.80000000', 'askPrice': '0.00034990', 'askQty': '174.59000000'}
{'symbol': 'XRPBTC', 'bidPrice': '0.00002616', 'bidQty': '3861.00000000', 'askPrice': '0.00002617', 'askQty': '29777.00000000'}
{'symbol': 'ADABTC', 'bidPrice': '0.00000469'

[========================================================================] 100%


No Trading Signal Triggered


[========================================================================] 100%


No Trading Signal Triggered


...

No Trading Signal Triggered


[========================================================================] 100%


No Trading Signal Triggered


[========================================================================] 100%


No Trading Signal Triggered


In [27]:
# test
import sys
import os
import pandas as pd
from binance.client import Client
import time
import progressbar
import matplotlib
from matplotlib import cm
import matplotlib.pyplot as plt
from binance.enums import *
import mpl_finance
# from BinanceKeys import BinanceKey1
# sys.path.append('C:\\Users\\Admin\\python-binance-master\\examples')

# os.chdir('C:/Users/Admin/python-binance-master/examples')
# sys.path.append('C:/Users/Admin/python-binance-master/examples')

import site
site.addsitedir(r'C:\\Users\\Admin\\python-binance-master\\examples')

import save_historical_data_Roibal
from BinanceKeys import BinanceKey1
# import technical_indicators

api_key = BinanceKey1['api_key']
api_secret = BinanceKey1['api_secret']

client = Client(api_key, api_secret)

# # get a deposit address for BTC
# address = client.get_deposit_address(asset='BTC')
# print(address)

watch_list=['ETHBTC','ADABTC','EOSBTC','XRPBTC']

trades = client.get_my_trades(symbol='ETHBTC')
asset_balance=client.get_asset_balance(asset='ETH')
print(type(asset_balance['free']))



<class 'str'>


In [41]:
klines=client.get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_30MINUTE, "30 Dec, 2017", "1 Jan, 2018")
ohlc=[]
for kline in klines:
    #print(kline)
    time1 = int(kline[0])
    open1 = float(kline[1])
    Low = float(kline[3])
    High = float(kline[2])
    Close = float(kline[4])
    Volume = float(kline[5])

ohlc.append([time1, open1, Close, High, Low, Volume])

print(ohlc)

[[1514764800000, 0.053586, 0.05375, 0.054042, 0.053474, 6805.697]]


In [57]:
import numpy as np
header=['Time','Open','High','Low','Close']
klines=client.get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_30MINUTE, "30 Dec, 2017", "1 Jan, 2018")
klines=np.array(klines)

klines_1=klines[:,0:5]
print(klines_1)
klines_2=pd.DataFrame(klines_1,columns=header)
print(klines_2)

[['1514592000000' '0.05102100' '0.05132500' '0.05100200' '0.05114900']
 ['1514593800000' '0.05114300' '0.05166700' '0.05090000' '0.05102900']
 ['1514595600000' '0.05094100' '0.05130000' '0.05083200' '0.05109400']
 ['1514597400000' '0.05109700' '0.05150000' '0.05068300' '0.05147000']
 ['1514599200000' '0.05147000' '0.05240400' '0.05000000' '0.05186400']
 ['1514601000000' '0.05188700' '0.05227800' '0.05151600' '0.05178900']
 ['1514602800000' '0.05174300' '0.05247200' '0.05151400' '0.05180400']
 ['1514604600000' '0.05180500' '0.05189800' '0.05123500' '0.05139800']
 ['1514606400000' '0.05148100' '0.05155000' '0.05080000' '0.05127000']
 ['1514608200000' '0.05127000' '0.05182000' '0.05112200' '0.05156000']
 ['1514610000000' '0.05156000' '0.05185200' '0.05140500' '0.05181000']
 ['1514611800000' '0.05181000' '0.05200000' '0.05158000' '0.05182200']
 ['1514613600000' '0.05187300' '0.05196100' '0.05150000' '0.05171000']
 ['1514615400000' '0.05173400' '0.05200000' '0.05161100' '0.05193400']
 ['151